In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re

In [2]:
# Cargamos los datos de entrenamiento
data_train = pd.read_csv("items_titles.csv")

# Cargamos los datos de prueba
data_test = pd.read_csv("items_titles_test.csv")


In [3]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
data_train.head(20)

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...
5,Tênis Casual Masculino Zarato 941 Preto 632
6,Tênis Infantil Ortopasso Conforto Jogging
7,Tv Samsung Qled 8k Q800t Semi Nova
8,Tênis Usthemp Short Temático - Maria Vira-lata 2
9,Sapatênis West Coast Urban Couro Masculino


In [4]:
data_test.head(20)

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde
5,Tênis Oakley Frequency 3.0 Preto/marrom
6,Tênis Jogging Feminino Premium Super Lançament...
7,Under Armour Hovr Phantom 2 Conexão Bluetooth ...
8,Tenis Infantil Feminino Menina Criança Moça
9,Tênis Labellamafia Saturn 6 Cores Disponíveis


In [5]:
#def corregir_palabra(palabra):
#    # Convertimos la palabra a minúsculas
#    palabra_min = palabra.lower()
#
#    # Creamos un diccionario de palabras similares
#    diccionario_palabras_similares = {
#        "tenis": "Tenis",
#        "teniss": "Tenis",
#        "tenisss": "Tenis",
#        "teni": "Tenis",
#        "tinnis": "Tenis",
#        "Tênis": "Tenis",
#        "tênis": "Tenis",
#        "tênis": "Tenis"
#    }
#
#    # Buscamos la palabra en el diccionario
#    palabra_encontrada = diccionario_palabras_similares.get(palabra_min)
#
#    # Si la palabra se encuentra en el diccionario, la corregimos
#    if palabra_encontrada:
#        return palabra_encontrada
#
#    # De lo contrario, devolvemos la palabra original
#    else:
#        return palabra
#
#def corregir_frase(frase):
#    # Convertimos la frase a minúsculas
#    frase_min = frase.lower()
#
#    # Iteramos sobre las palabras de la frase
#    for i in range(len(frase_min)):
#        # Obtenemos la palabra actual
#        palabra = frase_min[i]
#
#        # Corregimos la palabra
#        palabra_corregida = corregir_palabra(palabra)
#
#        # Reemplazamos la palabra actual por la palabra corregida
#        frase_min = frase_min[:i] + palabra_corregida + frase_min[i + 1:]
#
#    # Devolvemos la frase corregida
#    return frase_min
#
## Convertimos los títulos a minúsculas
#data_train["ITE_ITEM_TITLE"] = data_train["ITE_ITEM_TITLE"].str.lower()
#data_test["ITE_ITEM_TITLE"] = data_test["ITE_ITEM_TITLE"].str.lower()
#
## Eliminamos los espacios en blanco
#data_train["ITE_ITEM_TITLE"] = data_train["ITE_ITEM_TITLE"].str.strip()
#data_test["ITE_ITEM_TITLE"] = data_test["ITE_ITEM_TITLE"].str.strip()
#
#
## Aplicamos la función `corregir_frase()` a la variable
#data_train["ITE_ITEM_TITLE"] = data_train["ITE_ITEM_TITLE"].apply(corregir_frase)
#data_test["ITE_ITEM_TITLE"] = data_test["ITE_ITEM_TITLE"].apply(corregir_frase)
#

In [6]:
# Función para corregir todas las ocurrencias de palabras mal escritas para una palabra objetivo
# en una frase, dada un diccionario de palabras mal escritas.

def corregir_todas_las_palabras(frase, palabras_incorrectas, palabra_correcta):
  """
  Corrige todas las ocurrencias de palabras mal escritas para una palabra objetivo
  en una frase, dada un diccionario de palabras mal escritas.

  Args:
      frase: La frase a corregir.
      palabras_incorrectas: Un diccionario con las palabras mal escritas como claves
                          y la palabra correcta como valor.
      palabra_correcta: (Opcional) La palabra correcta para la que se corrigen las variantes.

  Returns:
      La frase corregida.
  """

  # Convertimos la frase a minúsculas
  frase_min = frase.lower()

  # Iteramos sobre cada palabra de la frase
  for i in range(len(frase_min)):
    # Obtenemos la palabra actual
    palabra = frase_min[i]

    # Comprobamos si la palabra actual está en el diccionario de palabras mal escritas
    if palabra in palabras_incorrectas:
      # La corregimos por la palabra correcta
      frase_min = frase_min[:i] + palabra_correcta + frase_min[i + 1:]

  # Devolvemos la frase corregida
  return frase_min

palabras_incorrectas = {
  "tenis": "Tenis",
  "zapatos": "Zapatos",
  "calzado": "Calzado",
  "tenis": "Tenis",
  "teniss": "Tenis",
  "tenisss": "Tenis",
  "teni": "Tenis",
  "tinnis": "Tenis",
  "Tênis": "Tenis",
  "tênis": "Tenis",
  "tênis": "Tenis"
}


## Convertimos los títulos a minúsculas
data_train["ITE_ITEM_TITLE"] = data_train["ITE_ITEM_TITLE"].str.lower()
data_test["ITE_ITEM_TITLE"] = data_test["ITE_ITEM_TITLE"].str.lower()
#
## Eliminamos los espacios en blanco
data_train["ITE_ITEM_TITLE"] = data_train["ITE_ITEM_TITLE"].str.strip()
data_test["ITE_ITEM_TITLE"] = data_test["ITE_ITEM_TITLE"].str.strip()
# Leer la base de datos y seleccionar la variable que contiene las frases

data_train['ITE_ITEM_TITLE'] = data_train['ITE_ITEM_TITLE'].str.strip()
data_train['ITE_ITEM_TITLE'] = data_train["ITE_ITEM_TITLE"].str.lstrip()

data_train["ITE_ITEM_TITLE_corregido"] = data_train["ITE_ITEM_TITLE"].apply(corregir_todas_las_palabras, palabras_incorrectas=palabras_incorrectas, palabra_correcta="Tenis")

data_train["ITE_ITEM_TITLE_corregido"] = data_train["ITE_ITEM_TITLE_corregido"].apply(corregir_todas_las_palabras, palabras_incorrectas=palabras_incorrectas, palabra_correcta="tenis")



# Definir una función para eliminar espacios en blanco al principio y al final usando expresiones regulares
def limpiar_espacios(cadena):
    return re.sub(r'^\s+|\s+$', '', cadena)

# Aplicar la función a la columna 'ITE_ITEM_TITLE'
data_train['ITE_ITEM_TITLE_corregido'] = data_train['ITE_ITEM_TITLE_corregido'].apply(limpiar_espacios)

# Mostrar el DataFrame después de la corrección
print(data_train)


                                          ITE_ITEM_TITLE  \
0      tênis ascension posh masculino - preto e vermelho   
1      tenis para caminhada super levinho spider corrida   
2      tênis feminino le parc hocks black/ice origina...   
3      tênis olympikus esportivo academia nova tendên...   
4      inteligente led bicicleta tauda luz usb bicicl...   
...                                                  ...   
29995  tênis vans old skool i love my vans - usado - ...   
29996                tênis feminino preto moleca 5296155   
29997  tenis botinha com pelo via marte original lanç...   
29998  tênis slip on feminino masculino original sapa...   
29999  bicicleta nathor rosa infantil sem pedal balan...   

                                ITE_ITEM_TITLE_corregido  
0      tênis ascension posh masculino - preto e vermelho  
1      tenis para caminhada super levinho spider corrida  
2      tênis feminino le parc hocks black/ice origina...  
3      tênis olympikus esportivo academia n

In [7]:
data_test.head(20)

,ITE_ITEM_TITLE
0,tênis olympikus esporte valente - masculino kids
1,bicicleta barra forte samy c/ 6 marchas cubo c...
2,tênis usthemp slip-on temático - labrador 2
3,tênis casual feminino moleca tecido tie dye
4,tênis star baby sapatinho conforto + brinde
5,tênis oakley frequency 3.0 preto/marrom
6,tênis jogging feminino premium super lançament...
7,under armour hovr phantom 2 conexão bluetooth ...
8,tenis infantil feminino menina criança moça
9,tênis labellamafia saturn 6 cores disponíveis


In [8]:
data_train.head(20)

,ITE_ITEM_TITLE,ITE_ITEM_TITLE_corregido
0,tênis ascension posh masculino - preto e vermelho,tênis ascension posh masculino - preto e vermelho
1,tenis para caminhada super levinho spider corrida,tenis para caminhada super levinho spider corrida
2,tênis feminino le parc hocks black/ice origina...,tênis feminino le parc hocks black/ice origina...
3,tênis olympikus esportivo academia nova tendên...,tênis olympikus esportivo academia nova tendên...
4,inteligente led bicicleta tauda luz usb bicicl...,inteligente led bicicleta tauda luz usb bicicl...
5,tênis casual masculino zarato 941 preto 632,tênis casual masculino zarato 941 preto 632
6,tênis infantil ortopasso conforto jogging,tênis infantil ortopasso conforto jogging
7,tv samsung qled 8k q800t semi nova,tv samsung qled 8k q800t semi nova
8,tênis usthemp short temático - maria vira-lata 2,tênis usthemp short temático - maria vira-lata 2
9,sapatênis west coast urban couro masculino,sapatênis west coast urban couro masculino


In [9]:
print(data_train["ITE_ITEM_TITLE_corregido"].str.lstrip())

0        tênis ascension posh masculino - preto e vermelho
1        tenis para caminhada super levinho spider corrida
2        tênis feminino le parc hocks black/ice origina...
3        tênis olympikus esportivo academia nova tendên...
4        inteligente led bicicleta tauda luz usb bicicl...
                               ...                        
29995    tênis vans old skool i love my vans - usado - ...
29996                  tênis feminino preto moleca 5296155
29997    tenis botinha com pelo via marte original lanç...
29998    tênis slip on feminino masculino original sapa...
29999    bicicleta nathor rosa infantil sem pedal balan...
Name: ITE_ITEM_TITLE_corregido, Length: 30000, dtype: object


In [10]:
def calculate_similarity(title_1, title_2):
    # Convertimos los títulos a vectores de características
    title_1_vec = np.array([word for word in title_1.split()])
    title_2_vec = np.array([word for word in title_2.split()])

    # Calculamos la similitud coseno
    return cosine_similarity(title_1_vec, title_2_vec)[0][0]


In [125]:
# Creamos una lista para almacenar las similitudes
similarities = []

# Iteramos sobre los datos de prueba
for title_1, title_2 in zip(data_train["ITE_ITEM_TITLE"].values, data_test["ITE_ITEM_TITLE"].values[1:]):
    # Calculamos la similitud
    similarity = calculate_similarity(title_1, title_2)

    # Agregamos la similitud a la lista
    similarities.append(similarity)


ValueError: could not convert string to float: 'tênis'

In [1]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from itertools import combinations
import numpy as np

titulos = ["zapatillas nike","Zapatos nike", "zapatillas nikke", "zapatillas nike grandes", "zapatillas", "tenis","tenis geo talla 2","Zapatos",'Ropa de bebe','Zapatos Jordan']

tokenized_titles = [word_tokenize(titulo.lower()) for titulo in titulos]

model = Word2Vec(tokenized_titles, vector_size=114, window=5, min_count=1, workers=6,epochs=175)

for titulo1, titulo2 in combinations(titulos, 2):
    palabras1 = word_tokenize(titulo1.lower())
    palabras2 = word_tokenize(titulo2.lower())

    similarity = model.wv.n_similarity(palabras1, palabras2)
    print(f"La similitud entre '{titulo1}' y '{titulo2}' es: {similarity}")


La similitud entre 'zapatillas nike' y 'Zapatos nike' es: 0.6216099262237549
La similitud entre 'zapatillas nike' y 'zapatillas nikke' es: 0.4947076439857483
La similitud entre 'zapatillas nike' y 'zapatillas nike grandes' es: 0.8369499444961548
La similitud entre 'zapatillas nike' y 'zapatillas' es: 0.7138404846191406
La similitud entre 'zapatillas nike' y 'tenis' es: -0.12841075658798218
La similitud entre 'zapatillas nike' y 'tenis geo talla 2' es: -0.021894939243793488
La similitud entre 'zapatillas nike' y 'Zapatos' es: 0.1273036003112793
La similitud entre 'zapatillas nike' y 'Ropa de bebe' es: -0.03839060664176941
La similitud entre 'zapatillas nike' y 'Zapatos Jordan' es: 0.019483188167214394
La similitud entre 'Zapatos nike' y 'zapatillas nikke' es: 0.13116997480392456
La similitud entre 'Zapatos nike' y 'zapatillas nike grandes' es: 0.4985864758491516
La similitud entre 'Zapatos nike' y 'zapatillas' es: 0.21758940815925598
La similitud entre 'Zapatos nike' y 'tenis' es: -0.07

In [3]:
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize
from itertools import combinations
import numpy as np

titulos = ["zapatillas nike", "zapatillas nikke", "ropa deportiva nike", "accesorios adidas", "zapatillas nike grandes", "zapatillas aquiles", "tenis geo", "tenis geo talla 2","ropa azil"]

tokenized_titles = [" ".join(word_tokenize(titulo.lower())) for titulo in titulos]

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

title_embeddings = []
for titulo in tokenized_titles:
    inputs = tokenizer(titulo, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    title_embeddings.append(embeddings)

for (titulo1, emb1), (titulo2, emb2) in combinations(zip(titulos, title_embeddings), 2):
    similarity = np.dot(emb1.flatten(), emb2.flatten()) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    print(f"La similitud entre '{titulo1}' y '{titulo2}' es: {similarity}")


La similitud entre 'zapatillas nike' y 'zapatillas nikke' es: 0.7846263647079468
La similitud entre 'zapatillas nike' y 'ropa deportiva nike' es: 0.6225166320800781
La similitud entre 'zapatillas nike' y 'accesorios adidas' es: 0.595533013343811
La similitud entre 'zapatillas nike' y 'zapatillas nike grandes' es: 0.7978516221046448
La similitud entre 'zapatillas nike' y 'zapatillas aquiles' es: 0.6896889209747314
La similitud entre 'zapatillas nike' y 'tenis geo' es: 0.4600256085395813
La similitud entre 'zapatillas nike' y 'tenis geo talla 2' es: 0.45129600167274475
La similitud entre 'zapatillas nike' y 'ropa azil' es: 0.5586099028587341
La similitud entre 'zapatillas nikke' y 'ropa deportiva nike' es: 0.580716073513031
La similitud entre 'zapatillas nikke' y 'accesorios adidas' es: 0.5868659019470215
La similitud entre 'zapatillas nikke' y 'zapatillas nike grandes' es: 0.7442139387130737
La similitud entre 'zapatillas nikke' y 'zapatillas aquiles' es: 0.5853251814842224
La similitud

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Descargar recursos adicionales de NLTK
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Cargar los datos
df_train = data_train
df_test = data_test

# Preprocesamiento de texto
def preprocess_text(text):
    stop_words = set(stopwords.words('portuguese'))
    stemmer = SnowballStemmer('portuguese')
    
    # Tokenización y eliminación de stop words
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    
    # Aplicar stemming
    tokens = [stemmer.stem(token) for token in tokens]
    
    return ' '.join(tokens)

# Aplicar preprocesamiento a los títulos
df_train['processed_title'] = df_train['ITE_ITEM_TITLE'].apply(preprocess_text)
df_test['processed_title'] = df_test['ITE_ITEM_TITLE'].apply(preprocess_text)

# Vectorización de texto usando TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix_train = vectorizer.fit_transform(df_train['processed_title'])
tfidf_matrix_test = vectorizer.transform(df_test['processed_title'])

# Calcular similitud de coseno
cosine_similarities = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)

# Definir un umbral de similitud
threshold = 0.5

# Crear un DataFrame para almacenar los resultados
results = []


## Iterar sobre los títulos de prueba y encontrar títulos similares en el conjunto de entrenamiento
#for i, test_title in enumerate(df_test['ITE_ITEM_TITLE']):
#    similar_titles = df_train[cosine_similarities[i] > threshold]['ITE_ITEM_TITLE'].tolist()
#    
#    # Calcular el score de similitud
#    similarity_scores = cosine_similarities[i][cosine_similarities[i] > threshold]
#    
#    # Escalar a un rango de 0 a 1
#    scaled_scores = (similarity_scores + 1) / 2
#    
#    results.append({'Test Title': test_title, 'Similar Titles': similar_titles, 'Similarity Scores': scaled_scores.tolist()})
#
#df_results = pd.DataFrame(results)
#


# Iterar sobre los títulos de prueba y encontrar títulos similares en el conjunto de entrenamiento
for i, test_title in enumerate(df_test['ITE_ITEM_TITLE']):
    similar_titles = df_train[cosine_similarities[i] > threshold]['ITE_ITEM_TITLE'].tolist()
    
    # Calcular el score de similitud
    similarity_scores = cosine_similarities[i][cosine_similarities[i] > threshold]
    
    # Escalar a un rango de 0 a 1
    scaled_scores = (similarity_scores + 1) / 2
    
    # Agregar cada par (título de prueba, título similar, score de similitud) como una nueva fila
    for similar_title, score in zip(similar_titles, scaled_scores):
        results.append({'Test Title': test_title, 'Similar Title': similar_title, 'Similarity Score': score})

        # Crear un DataFrame de resultados
df_results = pd.DataFrame(results)
df_results_sorted = df_results.sort_values(by='Similarity Score', ascending=False)

# Guardar los resultados en un archivo CSV
#df_results.to_csv('similarity_results.csv', index=False)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELLPHOTO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELLPHOTO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
df_results_sorted.head(100)

,Test Title,Similar Title,Similarity Score
269974,tênis casual kolosh feminino c2761,tênis casual feminino kolosh c1282,1.0
87048,tênis kolosh casual feminino - c1287,tênis casual feminino kolosh c1282,1.0
2924,tênis qix skate combat iv camurça preto branco...,tênis qix skate combat iv camurça preto branco...,1.0
270058,tênis casual kolosh feminino c2761,tênis kolosh c1629 feminino casual,1.0
87063,tênis kolosh casual feminino - c1287,tênis feminino kolosh k8681 casual,1.0
...,...,...,...
222712,tênis infantil led roller celebration de luz b...,tênis infantil led roller celebration de luz b...,1.0
222779,tênis polo farm 921 masculino - 2,tênis polo farm 275 masculino,1.0
222852,sapatenis masculino barato,sapatenis masculino barato,1.0
13440,tênis feminino casual leve dia a dia macio mod...,tênis feminino casual leve dia a dia macio mod...,1.0


In [13]:
df_results_sorted.tail(100)

,Test Title,Similar Title,Similarity Score
110548,tenis feminino chunky elastico tie dye,tênis slip on tie dye amarelo,0.750021
156080,tênis meia feminino masculino slip on academia...,tenis meias modare casual feminino slip on con...,0.750020
221553,sneaker chunky feminino pedraria removível cad...,tênis pedraria chunky sneaker feminino lançam...,0.750020
221551,sneaker chunky feminino pedraria removível cad...,tênis chunky pedraria sneaker feminino lançame...,0.750020
193307,tenis masculino para academia e caminhada,tenis activitta academia caminhada sem cadarço...,0.750019
...,...,...,...
60182,sapatênis colcci kelson em couro branco - masc...,sapatênis colcci lettering off-witte original,0.750001
152877,tênis masculino creation 20 caminhada envio im...,tênis wave creation 21,0.750001
203993,sapatenis tenis casual masculino solado reto r...,tênis sapatênis vermelho masculino + relógio d...,0.750000
147143,453072 tenis mini walk branco/marinho/vermelho...,tenis klin 09/2021 256.063 preto/goma,0.750000


In [18]:
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize
import numpy as np

def calculate_similarity(emb1, emb2):
    return np.dot(emb1.flatten(), emb2.flatten()) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

titulos_train = ["zapatillas nike", "ropa deportiva nike", "accesorios adidas", "zapatillas nike grandes", "tenis geo"]
titulos_test = ["zapatillas nikke", "zapatillas aquiles", "tenis geo talla 2", "ropa azil"]

tokenized_titles_train = [" ".join(word_tokenize(titulo.lower())) for titulo in titulos_train]
tokenized_titles_test = [" ".join(word_tokenize(titulo.lower())) for titulo in titulos_test]

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

title_embeddings_train = []
title_embeddings_test = []

for titulo in tokenized_titles_train:
    inputs = tokenizer(titulo, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    title_embeddings_train.append(embeddings)

for titulo in tokenized_titles_test:
    inputs = tokenizer(titulo, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    title_embeddings_test.append(embeddings)

# Calcular similitudes entre cada par de títulos (entrenamiento vs prueba)
for emb1, titulo1 in zip(title_embeddings_train, titulos_train):
    for emb2, titulo2 in zip(title_embeddings_test, titulos_test):
        similarity = calculate_similarity(emb1, emb2)
        print(f"La similitud entre '{titulo1}' y '{titulo2}' es: {similarity}")



La similitud entre 'zapatillas nike' y 'zapatillas nikke' es: 0.7846263647079468
La similitud entre 'zapatillas nike' y 'zapatillas aquiles' es: 0.6896890997886658
La similitud entre 'zapatillas nike' y 'tenis geo talla 2' es: 0.45129603147506714
La similitud entre 'zapatillas nike' y 'ropa azil' es: 0.5586095452308655
La similitud entre 'ropa deportiva nike' y 'zapatillas nikke' es: 0.5807158350944519
La similitud entre 'ropa deportiva nike' y 'zapatillas aquiles' es: 0.5997030138969421
La similitud entre 'ropa deportiva nike' y 'tenis geo talla 2' es: 0.5231325626373291
La similitud entre 'ropa deportiva nike' y 'ropa azil' es: 0.6328482031822205
La similitud entre 'accesorios adidas' y 'zapatillas nikke' es: 0.5868658423423767
La similitud entre 'accesorios adidas' y 'zapatillas aquiles' es: 0.5681068301200867
La similitud entre 'accesorios adidas' y 'tenis geo talla 2' es: 0.5963857173919678
La similitud entre 'accesorios adidas' y 'ropa azil' es: 0.6115542054176331
La similitud en

In [24]:
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd

def calculate_similarity(emb1, emb2):
    return np.dot(emb1.flatten(), emb2.flatten()) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

titulos_train = df_train['ITE_ITEM_TITLE'].sample(n=1000, random_state=42)
titulos_test = df_test['ITE_ITEM_TITLE'].sample(n=1000, random_state=42)

tokenized_titles_train = [" ".join(word_tokenize(titulo.lower())) for titulo in titulos_train]
tokenized_titles_test = [" ".join(word_tokenize(titulo.lower())) for titulo in titulos_test]

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

title_embeddings_train = []
title_embeddings_test = []

for titulo in tokenized_titles_train:
    inputs = tokenizer(titulo, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    title_embeddings_train.append(embeddings)

for titulo in tokenized_titles_test:
    inputs = tokenizer(titulo, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    title_embeddings_test.append(embeddings)

# Calcular similitudes y almacenar en un DataFrame
results = []

for emb1, titulo1 in zip(title_embeddings_train, titulos_train):
    for emb2, titulo2 in zip(title_embeddings_test, titulos_test):
        similarity = calculate_similarity(emb1, emb2)
        results.append({'Titulo 1': titulo1, 'Titulo 2': titulo2, 'Similarity': similarity})

# Crear DataFrame y ordenar por Similarity de mayor a menor
df_results = pd.DataFrame(results)
df_results_sorted = df_results.sort_values(by='Similarity', ascending=False)

# Mostrar el DataFrame ordenado
#print(df_results_sorted)


In [25]:
df_results_sorted.head(10)

,Titulo 1,Titulo 2,Similarity
534653,bicicleta feminina aro 26 na cor creme com cer...,bicicleta feminina aro 26 na cor creme com cer...,1.000000
321007,sapato infantil klin outdoor 161 - 1741602011,sapato infantil klin outdoor 161 - 1741602011,1.000000
527969,"tênis militar tático, esporte, caminhada, resi...","tênis militar tático, esporte, caminhada, resi...",1.000000
802060,tênis caminhada six street lunar - white green...,tênis caminhada six street lunar - white green...,1.000000
553300,tenis feminino masculino numero grande 38 39 4...,tenis feminino masculino numero grande 38 39 4...,1.000000
447427,bicicleta aro 16 com rodinhas feminina ultra kids,bicicleta com rodinhas aro 16 feminina ultra kids,0.993038
521048,tenis infantil masculino menino escolar c revenda,tenis infantil masculino menino escolar d revenda,0.985190
621459,sapatenis masculino em couro legítimo confort...,sapatenis masculino confortavel em couro legitimo,0.979148
382409,tênis usthemp half temático - chihuahua pelo ...,tênis usthemp move temático - chihuahua pelo ...,0.972723
172135,sapatenis feminino rosa confortável - compre a...,sapatenis feminino rosa cano baixo - compre ag...,0.972493


In [306]:
#from transformers import AutoModel, AutoTokenizer
!pip install transformers 


  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.9.0
    Uninstalling fsspec-0.9.0:
      Successfully uninstalled fsspec-0.9.0


In [310]:
pip install --upgrade typing-extensions

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
Note: you may need to restart the kernel to use updated packages.


In [308]:
!pip install sentence_transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125918 sha256=fcd767523fb391098ab6a5a5bbf3554582fa5bf6e5bd6fe3be691b60c70bfaef
  Stored in directory: c:\users\dellphoto\appdata\local\pip\cache\wheels\5e\6f\8c\d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
